- **find_element**
    - https://hyunsooworld.tistory.com/entry/%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%EC%98%A4%EB%A5%98-AttributeError-WebDriver-object-has-no-attribute-findelementbycssselector-%EC%98%A4%EB%A5%98%ED%95%B4%EA%B2%B0

In [ ]:
# !pip install selenium

### Import

In [11]:
import re
import time
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

### 📢 교육 카테고리 크롤링

#### 크롬 브라우저

In [ ]:
def Open_chrome(search:str):
    
    ''' 📢 원하는 검색어 입력해서 크롬 브라우저 열기 '''
    
    url = "https://map.naver.com/p/"
    
    # Chrome 드라이버를 생성
    driver = webdriver.Chrome()

    # 생성한 URL로 브라우저 열기
    driver.get(url)
    
    # 검색어 입력
    search_box = driver.find_element(By.CSS_SELECTOR, "#section_content > div > div.sc-iwm9f4.jCPpmH > div > div > button")
    time.sleep(1)
    search_box.send_keys("\n" + search + "\n")
    
    # iframe
    time.sleep(3)
    iframe_element = driver.find_element(By.ID, "searchIframe")
    driver.switch_to.frame(iframe_element)
    
    return driver

In [ ]:
driver = Open_chrome('서울학원')
driver

#### 스크롤 조정

In [ ]:
def scroll_container_by_amount(driver, container_element):
    
    ''' 📢 스크롤 끝까지 내리기 '''
    
    # 이전 스크롤 높이를 저장하는 변수를 초기화
    prev_scroll_height = 0
    
    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container_element)     # container_element를 아래로 스크롤
        time.sleep(1.5)
        
        curr_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", container_element)  # 현재 스크롤 높이를 가져옴
        
        # 현재 스크롤 높이와 이전 스크롤 높이를 비교하여 동일하다면 무한 루프를 종료
        if curr_scroll_height == prev_scroll_height:
            break
        
        # 현재 스크롤 높이를 이전 스크롤 높이로 업데이트
        prev_scroll_height = curr_scroll_height

#### 크롤링

In [ ]:
def Navermap_crawling(search:str, ele_tag:str, name_tag:str, catg_tag:str):
    
    """ 📢 네이버 지도의 마지막 페이지까지 크롤링 """
    
    driver = Open_chrome(search)
    
    # 데이터프레임 저장을 위해 빈 df 생성
    df = pd.DataFrame()
    previous_text = None  

    while True:
        
        # current_text : 현재 페이지 
        now_page = driver.find_element(By.CSS_SELECTOR, f"#app-root > div > div.XUrfU > div.zRM9F > a.mBN2s.qxokY")
        current_text = now_page.text

        if previous_text != current_text:
            container_element = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
            scroll_container_by_amount(driver, container_element)
            print('현재 페이지:', current_text)
            
            # find_elements(검색어에 따라 변경 필요) : 상호명, 카테고리
            element = driver.find_elements(By.CSS_SELECTOR, ele_tag)
            name = [i.find_element(By.CSS_SELECTOR, name_tag).text for i in element]  
            category = [i.find_element(By.CSS_SELECTOR, category_tag).text for i in element]    
            print(name)
            
            new_data = {'Name': name, 'Category': category, 'Search': search}
            df = df.append(pd.DataFrame(new_data), ignore_index=True)
            df.to_excel(f'./data/{search}_navermap_크롤링.xlsx', index=False)
            
            # 다음 페이지 클릭
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg").click()

        else:
            break
        
        # previous_text에 current_text(현재 페이지) 업데이트
        previous_text = current_text
        
    return df

#### 검색어 입력

In [ ]:
search = "서울학원"
ele_tag = "#_pcmap_list_scroll_container > ul > li > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1) > div > div"
name_tag = ".place_bluelink.YwYLL"
category_tag = ".YzBgS"

result_df = Navermap_crawling(search, element_tag, name_tag, category_tag)
result_df

In [ ]:
search = "취업학원"
ele_tag = "#_pcmap_list_scroll_container > ul > li > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1) > div > div"
name_tag = ".place_bluelink.YwYLL"
category_tag = ".YzBgS"

result_df = Navermap_crawling(search, element_tag, name_tag, category_tag)
result_df

#### 후처리

In [ ]:
data1 = pd.read_excel("./data/서울학원_navermap_크롤링.xlsx")
data2 = pd.read_excel("./data/취업학원_navermap_크롤링.xlsx")

In [ ]:
data = pd.concat([data1, data2])
data = data[(data['Category'] != "입시학원") & (data['Category'] != "교습학원,교습소") & (data['Category'] != "수학학원") & (data['Category'] != "학원,교습소")]
data = data.drop_duplicates(subset=['Name','Category'])
data

In [ ]:
# data.to_excel('./data/카테고리_교육_navermap_크롤링.xlsx', index=False)

### 📢 간편결제 가져오기
- 출처:https://namu.wiki/w/%EA%B0%84%ED%8E%B8%20%EA%B2%B0%EC%A0%9C%20%EC%84%9C%EB%B9%84%EC%8A%A4

In [49]:
pay_data = """
삼성전자 - 삼성 페이, 삼성 페이 mini[11]
LG전자 - LG 페이[12]
Apple - Apple Pay[13]
LG U+ - 페이나우[14]
SK그룹 - SK페이(11페이와 T페이의 통합 앱)
전자 소프트웨어 유통망
삼성전자 - Galaxy Store 및 Galaxy Themes에서 결제수단을 등록하여 스토어 내 및 정기 결제에서 간편 결제 서비스를 통해 결제 가능하며 인앱 구매 제외.[15]
구글 - Google Pay의 대한민국 내 Google Play용 서비스. Play 스토어 및 인앱 구매에서만 결제 가능.
원스토어 - 원페이
신용카드사(JUSTOUCH, 모든페이 포함)[16]
비씨카드 - 페이북/ISP
현대카드 - 현대카드(+앱카드)
삼성카드 - 삼성앱카드(JUSTOUCH 불가[17])
롯데카드 - 롯데카드 LIFE
KB국민카드 - KB Pay[18]
NH농협카드 - NH pay
신한카드 - 신한pLay[19]
한국씨티은행 - 씨티모바일(JUSTOUCH 불가)
하나카드 - 하나Pay(하나카드를 통해)[20]
은행/상호금융(ZEROPAY)
신한은행 - 신한 SOL
KB국민은행 - KB Pay
우리은행 - 우리WON뱅킹
중소기업은행 - i-ONE뱅크
부산은행 - sum뱅크
DGB대구은행 - iM뱅크
경남은행 - 투유뱅크개인
수협은행 - 수협파트너뱅크개인
NH농협은행, 농업협동조합 - NH앱캐시
케이뱅크 - 케이뱅크페이
MG새마을금고 - MG상상뱅크[21]
우체국예금 - 우체국페이
뱅크페이 앱 : 하나은행, KDB산업은행, SC제일은행, 전북은행, 광주은행, 제주은행, 우체국예금보험, MG새마을금고, 신용협동조합
정보매개 서비스사
네이버 - 네이버페이(N pay), 라인 페이(일본, 대만 등)
카카오 - 카카오페이 ## 카카오페이 정책 변경으로 30만 원 이상 결제시 일부 금액이 카카오페이머니로 리워드됨. (신한은 확인) 수정 시 참고 바람.
NHN - 페이코(PAYCO)
물류유통사/전자상거래 기업
롯데멤버스 - L.PAY(엘포인트를 통해)
SSG.COM - SSG PAY[22]
GS리테일 - GS PAY(우리동네GS를 통해)
CU - CU 간편결제(포켓CU를 통해)[23]
현대백화점그룹 - H.Point Pay
지마켓(G마켓/옥션) - 스마일페이[24]
쿠팡 - 쿠페이(Coupay)
위메프 - 위메프페이[25]
티몬 - 티몬페이(TMON Pay)
번개장터 - 번개페이
헬로마켓 - 헬로페이
당근 - 당근페이
와디즈[26]
교보문고 - 교보페이[27]
예스24 - 세이페이
SPC - 해피페이(해피포인트, 해피오더, 파리바게뜨를 통해)[28]
파리바게뜨[29]
컴퓨존 - 컴퓨존페이[30]
핀테크/교통카드 기업
KG이니시스 - 케이페이(Kpay)
비바리퍼블리카 - 토스페이 (토스를 통해)
한국NFC - 페이앱NFC, 페이앱라이트
차이코퍼레이션 - 차이페이
티머니 - 티머니페이
기타업체
미래에셋증권 - 미래에셋페이[31]
현대자동차 - 현대 카페이, 제네시스 카페이[A]
기아 - 기아페이[A]
티웨이항공 - 티웨이페이[34]
배달의 민족 - 배민페이
"""

In [50]:
# 데이터를 줄바꿈 문자('\n')를 기준으로 분할하여 리스트로 변환
data_lines = pay_data.strip().split('\n')

# 데이터를 딕셔너리로 구성한 후 데이터프레임으로 변환
data_dict = {'회사/기관': [], '결제 서비스': []}

for line in data_lines:
    parts = line.split('-')
    if len(parts) == 2:
        company = parts[0].strip()
        services = parts[1].strip()
        data_dict['회사/기관'].append(company)
        data_dict['결제 서비스'].append(services)

df = pd.DataFrame(data_dict)
df

,회사/기관,결제 서비스
0,삼성전자,"삼성 페이, 삼성 페이 mini[11]"
1,LG전자,LG 페이[12]
2,Apple,Apple Pay[13]
3,LG U+,페이나우[14]
4,SK그룹,SK페이(11페이와 T페이의 통합 앱)
5,삼성전자,Galaxy Store 및 Galaxy Themes에서 결제수단을 등록하여 스토어 ...
6,구글,Google Pay의 대한민국 내 Google Play용 서비스. Play 스토어 ...
7,원스토어,원페이
8,비씨카드,페이북/ISP
9,현대카드,현대카드(+앱카드)


#### 후처리

In [56]:
df = df.drop_duplicates(subset=['회사/기관'], keep='first')
df['결제 서비스'] = df['결제 서비스'].apply(lambda x: re.sub(r'\[\d+\]', '', x).strip())
df['결제 서비스'] = df['결제 서비스'].apply(lambda x: re.sub(r'\[[Aa]\]', '', x).strip())
df['결제 서비스'] = df['결제 서비스'].apply(lambda x: re.sub(r'\(.*\)', '', x).strip())
df['결제 서비스'] = df['결제 서비스'].apply(lambda x: x.split('##')[0])
df.loc[df['회사/기관'] == "구글", '결제 서비스'] = "Google Pay"

df['결제 서비스'] = df['결제 서비스'].str.split(',').str[0]
df

,회사/기관,결제 서비스
0,삼성전자,삼성 페이
1,LG전자,LG 페이
2,Apple,Apple Pay
3,LG U+,페이나우
4,SK그룹,SK페이
6,구글,Google Pay
7,원스토어,원페이
8,비씨카드,페이북/ISP
9,현대카드,현대카드
10,삼성카드,삼성앱카드


In [57]:
# df.to_excel('./data/카테고리_간편결제_navermap_크롤링.xlsx', index=False)